In [1]:
import os
import json
import traceback
from datetime import datetime
from flask import Flask, render_template_string, request, redirect, url_for
from io import BytesIO
import base64
import matplotlib.pyplot as plt

app = Flask(__name__)

# --- long-term storage for player directory with stats ---
DB_FILE = 'players_db.json'
# --- storage for completed games ---
GAMES_FILE = 'games_db.json'

In [ ]:
# --- base HTML head/footer with light-blue Bootstrap theme ---
tpl_base_head = '''
<!doctype html><html lang="en"><head><meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
<title>Pokerzone</title>
<style>
  body { background: #e0f4ff; }
  .card { background: #cceaff; border-color: #99d6ff; }
  .btn-primary { background: #99d6ff; border-color: #99d6ff; }
  .btn-primary:hover { background: #80c2ff; }
  .btn-secondary { background: #eef5fc; border-color: #99d6ff; color: #003f5c; }
  .btn-secondary:hover { background: #d9ecff; }
  .list-group-item { background: #cceaff; border-color: #99d6ff; }
  .badge { background: #99d6ff; color: #fff; }
</style></head><body><div class="container py-4">
'''
base_footer = '''
</div>
</body>
</html>
'''

# Index template with Stats button
tpl_index = tpl_base_head + '''
<div class="card mb-4"><div class="card-body">
  <h1 class="text-primary">Pokerzone</h1>
  {% if active %}
    <span class="badge mb-2">In Progress</span><br>
    <a href="{{ url_for('add_player') }}" class="btn btn-primary me-2">Add Player</a>
    <a href="{{ url_for('finalize_game') }}" class="btn btn-primary me-2">End Game</a>
    <a href="{{ url_for('view_stats') }}" class="btn btn-secondary">Stats</a>
  {% else %}
    <a href="{{ url_for('start_game') }}" class="btn btn-primary me-2">Start New Game</a>
    <a href="{{ url_for('view_stats') }}" class="btn btn-secondary">Stats</a>
  {% endif %}
</div></div>

{% if active %}
<div class="row">
  <div class="col-md-6">
    <div class="card mb-4">
      <div class="card-header text-primary"><h5>Players</h5></div>
      <ul class="list-group list-group-flush">
        {% for p in players %}
        <li class="list-group-item d-flex justify-content-between align-items-center">
          {{ p.name }} – ${{ '%.2f'|format(p.buyin) }}
          <div>
            {% if p.final is none %}
              <a href="{{ url_for('addon', idx=loop.index0) }}" class="btn btn-sm btn-secondary me-1">Add-On</a>
              <a href="{{ url_for('cashout', idx=loop.index0) }}" class="btn btn-sm btn-secondary">Cash-Out</a>
            {% else %}
              <span class="badge">Cashed: ${{ '%.2f'|format(p.final) }}</span>
            {% endif %}
          </div>
        </li>
        {% endfor %}
      </ul>
    </div>
  </div>
  <div class="col-md-6">
    <div class="card mb-4">
      <div class="card-header text-primary"><h5>Action Log</h5></div>
      <ul class="list-group list-group-flush">
        {% for entry in log %}
        <li class="list-group-item">{{ entry|safe }}</li>
        {% endfor %}
      </ul>
    </div>
  </div>
</div>
{% endif %}
''' + '''
</div></body></html>'''

# --- add-player form (select existing or new) ---
tpl_add = tpl_base_head + '''
<div class="card mx-auto" style="max-width:500px;">
  <div class="card-header text-primary"><h5>Add Player</h5></div>
  <div class="card-body">
    <form method="post">
      <div class="mb-3">
        <label class="form-label">Choose Existing Player</label>
        <select name="dir_idx" class="form-select">
          <option value="new" selected>-- Add New Player --</option>
          {% for i, p in players_db %}
            <option value="{{ i }}">{{ p['name'] }}</option>
          {% endfor %}
        </select>
      </div>
      <div class="mb-3">
        <label class="form-label">Name</label>
        <input name="name" class="form-control">
      </div>
      <div class="mb-3">
        <label class="form-label">Phone</label>
        <input name="phone" class="form-control">
      </div>
      <div class="mb-3">
        <label class="form-label">BSB</label>
        <input name="bsb" class="form-control">
      </div>
      <div class="mb-3">
        <label class="form-label">Account #</label>
        <input name="account" class="form-control">
      </div>
      <div class="mb-3">
        <label class="form-label">Buy-in</label>
        <input name="buyin" type="number" step="0.01" class="form-control" required>
      </div>
      <button class="btn btn-primary">Add</button>
      <a href="{{ url_for('index') }}" class="btn btn-secondary ms-2">Cancel</a>
    </form>
  </div>
</div>
''' + base_footer

# --- add-on form ---
tpl_addon = tpl_base_head + '''
<div class="card mx-auto" style="max-width:400px;">
  <div class="card-header text-primary"><h5>Add-On for {{ player.name }}</h5></div>
  <div class="card-body">
    <form method="post">
      <div class="mb-3"><label class="form-label">Amount</label>
        <input name="addon" type="number" step="0.01" class="form-control" required>
      </div>
      <button class="btn btn-primary">Submit</button>
      <a href="{{ url_for('index') }}" class="btn btn-secondary ms-2">Back</a>
    </form>
  </div>
</div>
''' + base_footer

# --- cash-out form ---
tpl_cashout = tpl_base_head + '''
<div class="card mx-auto" style="max-width:400px;">
  <div class="card-header text-primary"><h5>Cash-Out for {{ player.name }}</h5></div>
  <div class="card-body">
    <form method="post">
      <div class="mb-3"><label class="form-label">Final Amount</label>
        <input name="final" type="number" step="0.01" class="form-control" required>
      </div>
      <button class="btn btn-primary">Submit</button>
      <a href="{{ url_for('index') }}" class="btn btn-secondary ms-2">Back</a>
    </form>
  </div>
</div>
''' + base_footer

# --- finalize form ---
tpl_finalize = tpl_base_head + '''
<div class="card mx-auto" style="max-width:600px;">
  <div class="card-header text-primary"><h5>Finalize Game</h5></div>
  <div class="card-body">
    <form method="post">
      {% for p in players %}
        {% if p.final is none %}
        <div class="mb-3">
          <label class="formlabel">{{ p.name }} Final Amount</label>
          <input name="final_{{ loop.index0 }}" type="number" step="0.01" class="formcontrol" required>
        </div>
        {% endif %}
      {% endfor %}
      <button class="btn btn-primary">Finish</button>
      <a href="{{ url_for('index') }}" class="btn btnsecondary ms-2">Cancel</a>
    </form>
  </div>
</div>
''' + base_footer

# --- results page ---
tpl_results = tpl_base_head + """
<div class="card mb-4">
  <div class="card-header text-primary">
    <h5>Game Results</h5>
  </div>
  <div class="card-body">
    <h6>Transfers</h6>
    <ul class="list-group mb-4">
      {% for t in transfers %}
        <li class="list-group-item">{{ t }}</li>
      {% endfor %}
    </ul>

    <h6>Action Log</h6>
    <ul class="list-group mb-4">
      {% for entry in log %}
        <li class="list-group-item">{{ entry|safe }}</li>
      {% endfor %}
    </ul>

    <div class="d-flex">
      <a href="{{ url_for('start_game') }}" class="btn btn-primary me-2">Start New Game</a>
      <a href="{{ url_for('index') }}" class="btn btn-secondary">Back to Lobby</a>
    </div>
  </div>
</div>
""" + base_footer

# Stats page template
tpl_stats = tpl_base_head + """
<div class="card mb-4">
  <div class="card-header text-primary"><h5>Player Statistics</h5></div>
  <div class="card-body">
    <h6>Net Stack by Player</h6>
    <img src="data:image/png;base64,{{ stack_chart }}" class="img-fluid mb-4"/>

    <h6>Time Played by Player</h6>
    <img src="data:image/png;base64,{{ time_chart }}" class="img-fluid mb-4"/>

    <h6 class="mt-4">Details</h6>
    <table class="table table-striped">
      <thead>
        <tr>
          <th>Name</th>
          <th>Net Stack</th>
          <th>Time Played</th>
          <th>Contact</th>
        </tr>
      </thead>
      <tbody>
        {% for p in stats %}
        <tr>
          <td>{{ p.name }}</td>
          <td>${{ "%.2f"|format(p.net_stack) }}</td>
          <td>{{ p.time_display }}</td>
          <td>{{ p.phone }}<br>{{ p.bsb }}/{{ p.account }}</td>
        </tr>
        {% endfor %}
      </tbody>
    </table>
    <a href="{{ url_for('index') }}" class="btn btn-secondary">← Back</a>
  </div>
</div>
""" + base_footer

In [3]:
if os.path.exists(DB_FILE):
    with open(DB_FILE) as f:
        players_db = json.load(f)
else:
    players_db = []  # list of {name, phone, bsb, account, net_stack, time_played_seconds}


if os.path.exists(GAMES_FILE):
    with open(GAMES_FILE) as f:
        games_db = json.load(f)
else:
    games_db = []  # list of {date, net_changes, time_played_seconds}

# --- current game state ---
game_active = False
players = []    # each: {name, phone, bsb, account, buyin, final, join_time, cashout_time}
actions = []    # list of HTML strings with timestamps
game_start_time = None

# --- logging helper ---
def log_action(text):
    ts = datetime.now().strftime('%H:%M')
    entry = f"[{ts}] {text}"
    actions.append(entry)

# --- debt settlement algorithm ---
def settle_debts(nets):
    creditors, debtors = [], []
    for name, net in nets.items():
        if net > 0:
            creditors.append([name, net])
        elif net < 0:
            debtors.append([name, -net])
    creditors.sort(key=lambda x: x[1], reverse=True)
    debtors.sort(key=lambda x: x[1], reverse=True)

    transfers = []
    i = j = 0
    while i < len(debtors) and j < len(creditors):
        d_name, d_amt = debtors[i]
        c_name, c_amt = creditors[j]
        amt = min(d_amt, c_amt)
        transfers.append((d_name, c_name, amt))
        debtors[i][1] -= amt
        creditors[j][1] -= amt
        if debtors[i][1] == 0:
            i += 1
        if creditors[j][1] == 0:
            j += 1
    return transfers

In [4]:
# --- global error handler ---
@app.errorhandler(Exception)
def handle_exception(e):
    return f"<pre>{traceback.format_exc()}</pre>", 500

# --- routes ---
@app.route('/')
def index():
    return render_template_string(tpl_index,
        active=game_active, players=players, log=actions)

@app.route('/start')
def start_game():
    global game_active, players, actions, game_start_time
    game_active = True
    players = []
    actions = []
    game_start_time = datetime.now()
    log_action('Game started')
    return redirect(url_for('index'))

@app.route('/add_player', methods=['GET','POST'])
def add_player():
    if not game_active:
        return redirect(url_for('index'))

    if request.method == 'POST':
        dir_idx = request.form['dir_idx']
        if dir_idx != 'new':
            idx = int(dir_idx)
            rec = players_db[idx]
            name, phone, bsb, account = rec['name'], rec['phone'], rec['bsb'], rec['account']
        else:
            name  = request.form['name']
            phone = request.form.get('phone','')
            bsb   = request.form.get('bsb','')
            account = request.form.get('account','')
            # persist new
            players_db.append({'name':name,'phone':phone,'bsb':bsb,'account':account,'net_stack':0,'timeplayed_seconds':0})
            with open(DB_FILE,'w') as f:
                json.dump(players_db, f, indent=2)

        buyin = float(request.form['buyin'])
        players.append({
          'name': name, 'phone': phone, 'bsb': bsb, 'account': account,
          'buyin': buyin, 'final': None,
          'join_time': datetime.now(), 'cashout_time': None
        })
        pot = sum(p['buyin'] for p in players)
        log_action(f"{name} bought in $ {buyin:.2f}  <span class='floatend'>Pot: $ {pot:.2f}</span>")
        return redirect(url_for('index'))

    return render_template_string(tpl_add, players_db=list(enumerate(players_db)))

@app.route('/stats')
def view_stats():
    # 1) Load and aggregate all finished games
    try:
        with open('games_db.json') as f:
            all_games = json.load(f)
    except FileNotFoundError:
        all_games = []

    totals = {}
    for game in all_games:
        # game['net_changes']: { name: net_change }
        # game['time_played']:  { name: seconds_played }
        for name, net in game.get('net_changes', {}).items():
            rec = totals.setdefault(name, {'net_stack': 0.0, 'time_played_seconds': 0})
            rec['net_stack'] += net
        for name, secs in game.get('time_played', {}).items():
            rec = totals.setdefault(name, {'net_stack': 0.0, 'time_played_seconds': 0})
            rec['time_played_seconds'] += secs

    # 2) Load directory for contact info
    try:
        with open(DB_FILE) as f:
            directory = { r['name']: r for r in json.load(f) }
    except FileNotFoundError:
        directory = {}

    # 3) Build sorted stats list
    stats = []
    for name, data in totals.items():
        # format hh:mm
        secs = data['time_played_seconds']
        hh, mm = divmod(secs, 3600)[0], divmod(secs % 3600, 60)[0]
        # get contact
        rec = directory.get(name, {})
        stats.append({
            'name': name,
            'net_stack': data['net_stack'],
            'time_display': f"{hh:02d}:{mm:02d}",
            'phone': rec.get('phone', ''),
            'bsb':   rec.get('bsb', ''),
            'account': rec.get('account', ''),
        })

    # sort by net_stack descending
    stats.sort(key=lambda p: p['net_stack'], reverse=True)

    # 4) Prepare charts
    names  = [p['name'] for p in stats]
    stacks = [p['net_stack'] for p in stats]
    times  = [int(p['time_display'].split(':')[0]) * 3600 +
              int(p['time_display'].split(':')[1]) * 60
              for p in stats]

    # Net Stack chart
    fig, ax = plt.subplots()
    ax.bar(names, stacks)
    ax.set_title('Net Stack by Player')
    ax.set_ylabel('Net Stack ($)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    buf = BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig)
    stack_chart = base64.b64encode(buf.getvalue()).decode('ascii')

    # Time Played chart
    fig2, ax2 = plt.subplots()
    ax2.bar(names, times)
    ax2.set_title('Time Played by Player')
    ax2.set_ylabel('Seconds')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    buf2 = BytesIO()
    fig2.savefig(buf2, format='png')
    plt.close(fig2)
    time_chart = base64.b64encode(buf2.getvalue()).decode('ascii')

    # 5) Render
    return render_template_string(
        tpl_stats,
        stats=stats,
        stack_chart=stack_chart,
        time_chart=time_chart
    )

@app.route('/addon/<int:idx>', methods=['GET','POST'])
def addon(idx):
    player = players[idx]
    if request.method == 'POST':
        amt = float(request.form['addon'])
        player['buyin'] += amt
        pot = sum(p['buyin'] for p in players)
        log_action(f"{player['name']} added on $ {amt:.2f}  <span class='floatend'>Pot: $ {pot:.2f}</span>")
        return redirect(url_for('index'))
    return render_template_string(tpl_addon, player=player)

@app.route('/cashout/<int:idx>', methods=['GET','POST'])
def cashout(idx):
    player = players[idx]
    if request.method == 'POST':
        final = float(request.form['final'])
        player['final'] = final
        player['cashout_time'] = datetime.now()
        total_buyin = sum(p['buyin'] for p in players)
        total_cashed = sum((p['final'] or 0) for p in players)
        remaining = total_buyin - total_cashed
        log_action(f"{player['name']} cashed out $ {final:.2f}  <span class='floatend'>Pot: $ {remaining:.2f}</span>")
        return redirect(url_for('index'))
    return render_template_string(tpl_cashout, player=player)

@app.route('/finalize', methods=['GET','POST'])
def finalize_game():
    global game_active
    # Prevent finalizing when no game active or no players
    if not game_active:
        return redirect(url_for('index'))
    if not players:
        log_action('No players to finalize. Game reset.')
        game_active = False
        return redirect(url_for('index'))
    if request.method == 'POST':
        finalize_time = datetime.now()
        # collect missing finals
        for i, p in enumerate(players):
            if p['final'] is None:
                p['final'] = float(request.form[f'final_{i}'])
                p['cashout_time'] = finalize_time
                log_action(f"{p['name']} final $ {p['final']:.2f}")

        buyin_sum = sum(p['buyin'] for p in players)
        cash_sum  = sum(p['final'] for p in players)
        diff = round(cash_sum - buyin_sum, 2)

        nets = {p['name']: p['final'] - p['buyin'] for p in players}
        if diff != 0:
            winner = max(nets, key=nets.get)
            nets[winner] -= diff
            log_action(f"Discrepancy of $ {diff:.2f} adjusted from {winner}")

        # settle debts
        raw = settle_debts(nets)
        # format transfers with bsb/account
        transfers = []
        for d, c, amt in raw:
            pd = next(item for item in players if item['name']==d)
            pc = next(item for item in players if item['name']==c)
            transfers.append(
              f"{d} ({pd['bsb']}/{pd['account']}) pays "
              f"{c} ({pc['bsb']}/{pc['account']}): ${amt:.2f}"
            )

        # record game in games_db
        game_record = {
            'date': game_start_time.strftime('%Y-%m-%d'),
            'net_changes': {p['name']: p['final'] - p['buyin'] for p in players},
            'time_played_seconds': {
                p['name']: int(((p['cashout_time'] or finalize_time) - p['join_time']).total_seconds())
                for p in players
            }
        }
        games_db.append(game_record)
        with open(GAMES_FILE, 'w') as f:
            json.dump(games_db, f, indent=2)

        # update player stats
        for p in players:
            rec = next(item for item in players_db if item['name']==p['name'])
            rec['net_stack'] = rec.get('net_stack', 0) + (p['final'] - p['buyin'])
            rec['time_played_seconds'] = rec.get('time_played_seconds', 0) + int(((p['cashout_time'] or finalize_time) - p['join_time']).total_seconds())
        with open(DB_FILE, 'w') as f:
            json.dump(players_db, f, indent=2)

        log_action('Game ended')
        game_active = False
        return render_template_string(tpl_results, transfers=transfers, log=actions)

    return render_template_string(tpl_finalize, players=players)

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/May/2025 12:40:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2025 12:40:54] "GET /favicon.ico HTTP/1.1" 500 -
127.0.0.1 - - [27/May/2025 12:40:55] "GET /stats HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2025 12:40:56] "GET /favicon.ico HTTP/1.1" 500 -
